<a href="https://colab.research.google.com/github/scecilia028/5gflix-data-analytics/blob/main/Notebooks/desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio 5GFlix - Engenharia de Dados - Solvimm**

Análise de filmes e séries da Netflix para estratégia de negócio.


Etapas do processo para codificação e resolução das respostas:


*   ETL
*   Resolução em ordem das questões para análise 




### Importação dos datasets

In [1]:
import pandas as pd
import csv

urlMovies = 'https://raw.githubusercontent.com/scecilia028/5gflix-data-analytics/main/Dados/movies.csv'
table_movies = pd.read_csv(urlMovies, sep='[;,]', engine='python', names=('Movie_Id', 'Title', 'YearRelease'))
table_movies['Title'] = table_movies['Title'].str[1:]
table_movies['YearRelease'] = table_movies['YearRelease'].str[:-1]

display(table_movies)

#input_movies = open('/content/drive/MyDrive/../movies.csv', 'r')
#fix_movies = open('fixmovies.csv', 'w')
#csv.register_dialect('dialect',
                     #delimiter=';',
                     #skipinitialspace=True,
                     #quoting=csv.QUOTE_ALL)
#movies = csv.reader(input_movies, dialect= 'dialect')
#writer = csv.writer(fix_movies,  delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL) 
 
#for row in movies: 
#  line_film_year = row[1].split(',')
#  title = line_film_year[0][1:]
#  year = line_film_year[1][:-1]
#  writer.writerow([row[0], title, year])

#input_movies.close()
#fix_movies.close()

#table_movies = pd.read_csv('fixmovies.csv', sep =',', names=('Movie_Id', 'Title', 'YearRelease'))
#display(table_movies)

,Movie_Id,Title,YearRelease
0,1,Dinosaur Planet,2003
1,2,Isle of Man TT 2004 Review,2004
2,3,Character,1997
3,4,Paula Abdul's Get Up & Dance,1994
4,5,The Rise and Fall of ECW,2004
...,...,...,...
4494,4495,Clifford: Happy Birthday Clifford / Puppy Love,2002
4495,4496,Farewell My Concubine,1993
4496,4497,Texasville,1990
4497,4498,Gonin,2000


*Importação do arquivo customers_rating.csv*

In [2]:
customers_rating = pd.read_csv('customers_rating.csv', delimiter=';')
display(customers_rating)

FileNotFoundError: ignored

### Limpeza dos dados
*Limpeza de dados duplicados e incompletos.*

In [199]:
#deletando linhas com pelo menos 1 valor NaN
table_movies = table_movies.dropna()
customers_rating = customers_rating.dropna()

#deletando dados com titulos de filmes replicados
moviesRp = table_movies.groupby('Title')['YearRelease'].value_counts().sort_values(ascending=False)
display(moviesRp)
#'Prova' - o filme 'Dr. Quinn' tem dois lançamentos em 1993
#movie_Quinn = table_movies.loc[table_movies['Title'] == 'Dr. Quinn']
#display(movie_Quinn)

table_movies = table_movies.drop_duplicates(subset =['Title','YearRelease'], keep='first')
customers_rating = customers_rating.drop_duplicates(keep='first')
#merge com ambas tabelas
tb_movie_rating = pd.merge(table_movies, customers_rating, how = 'outer')

Title                                                               YearRelease
'N Sync: 'N the Mix                                                  1999          1
Shaka Zulu                                                           1986          1
Shakespeare Wallah                                                   1965          1
Shakespeare Tragedies: Titus/ Romeo/ Hamlet: The Standard Deviants   2000          1
Shakespeare Tragedies: Othello                                       1981          1
                                                                                  ..
Harold and Kumar Go to White Castle                                  2004          1
Harper Valley P.T.A.                                                 1978          1
Harrison's Flowers                                                   2002          1
Harry Potter and the Prisoner of Azkaban: Bonus Material             2004          1
s-Cry-ed                                                             2

# Resolução das questões 
 
 *Questão 1.1 - Quantidade de filmes disponíveis no DataSet*

In [5]:
#mais rapido
print(len(table_movies))
#modo mais lento, porém mais viável se houvesse NaN
print(table_movies[table_movies.columns[0]].count())


4499
4499


*Questão 1.2 - nome dos 5 filmes com melhor média de avaliação*

In [6]:
top5_rating = tb_movie_rating.groupby(['Title'])['Rating'].agg(['mean']).sort_values(by='mean', ascending=False).head(5)
display(top5_rating)

,mean
Title,
Lost: Season 1,4.670989
Ghost in the Shell: Stand Alone Complex: 2nd Gig,4.586364
The Simpsons: Season 6,4.581296
Inu-Yasha,4.554434
Lord of the Rings: The Return of the King: Extended Edition: Bonus Material,4.552000


*Questão 1.3 - 5 anos com menos lançamentos de filmes*

In [7]:
#por ordem de lançamento
top5_lessYearRelease = table_movies['YearRelease'].sort_values(ascending=True).value_counts(sort=False).sort_values(ascending=True).head(5)
display(top5_lessYearRelease)


1915    1
1917    1
1922    1
1926    1
1916    2
Name: YearRelease, dtype: int64

*Questão 1.4 - filmes que possuem avaliação maior ou igual a 4.7, considerando apenas os filmes avaliados na última data de avaliação do dataset*


In [116]:
#pensamento lógico: existem varias avaliacoes para o mesmo filme na mesma data, então, usei media

#%%timeit
#lastDate = tb_movie_rating['Date'].sort_values(ascending=True).loc[1]
#1 loop, best of 5: 53 s per loop

#%%timeit
lastDate = tb_movie_rating['Date'].max(skipna = True)
#1 loop, best of 5: 1.72 s per loop

#ultima data é 2005-12-31
tb_moviesRLD = tb_movie_rating.loc[(tb_movie_rating['Date'] == lastDate)]
tb_meanRatingLastDate = tb_moviesRLD.groupby('Title')['Rating'].mean()
tb_moviesRatingLastDate = tb_meanRatingLastDate.loc[(tb_meanRatingLastDate >= 4.7)]
tb_moviesRatingLastDate.index.tolist()

 

['2002 Olympic Figure Skating Competition',
 'A Better Tomorrow II',
 'A Far Off Place',
 'A Night at the Opera',
 'Absolutely Fabulous: Series 2',
 'Ah! My Goddess: The Movie',
 'Alias: Season 1',
 'American Gothic: The Complete Series',
 'American Ninja',
 'An Evening With Kevin Smith',
 'Angel: Season 4',
 'Another You',
 'Armed and Dangerous',
 'Attack of the Killer Tomatoes',
 'Baby Einstein: Baby Monet: Discovering the Seasons',
 'Baby Neptune: Discovering Water',
 'Babylon 5: Season 4',
 "Backyardigans: It's Great to Be a Ghost",
 'Batman the Animated Series: Vol. 3',
 'Batman: Mask of the Phantasm',
 'Batman: The Animated Series: Tales of the Dark Knight',
 'Bear Cub',
 'Beau Travail',
 'Benny Hill: Golden Classics',
 'Beyonce: Live at Wembley',
 'Bjork: Cambridge',
 'Bjork: Shepherds Bush Empire',
 'Bjork: The Inner or Deep Part of an Animal or Plant Structure',
 'Bob the Builder: Teamwork',
 'Boy Meets World: Season 1',
 'Bringing Up Baby',
 'Charlie Chan: The Shanghai Cobra'

*Questão 1.5 - Dos filmes encontrados na questão anterior, quais são os 10 filmes com as piores notas e
quais as notas*

In [117]:
#levando totalmente em conta o enunciado da questão anterior
tb_moviesWorstRating = tb_moviesRatingLastDate.sort_values(ascending=True).head(10)
display(tb_moviesWorstRating)
print('\n Outra opção \n')
#levando em conta somente a última data da questão anterior
#Ou seja, os filmes e notas com as piores notas levando em conta apenas o último dia de avaliacao registrado
tb_moviesWorstRatingDate = tb_meanRatingLastDate.sort_values(ascending=True).head(10)
display(tb_moviesWorstRatingDate)

Title
Spirited Away                      4.714286
Gilmore Girls: Season 3            4.750000
The Twilight Zone: Vol. 15         4.750000
SpongeBob SquarePants: Season 3    4.750000
Pete's Dragon                      4.750000
Absolutely Fabulous: Series 2      4.750000
The Twilight Zone: Vol. 41         4.750000
An Evening With Kevin Smith        4.750000
Curb Your Enthusiasm: Season 3     4.800000
In the Mood for Love               4.800000
Name: Rating, dtype: float64


 Outra opção 



Title
The Muse                                                1.0
Fade to Black                                           1.0
Grand Theft Parsons                                     1.0
Commanding Heights: The Battle for the World Economy    1.0
Convict 762                                             1.0
Cop and a Half                                          1.0
I Love Lucy: Season 2                                   1.0
Angel Sanctuary                                         1.0
The Closet                                              1.0
Slam                                                    1.0
Name: Rating, dtype: float64

*Questão 1.6 - id's dos 5 customer que mais avaliaram filmes e quantas avaliações cada um
fez?*

In [118]:
#com pivot
tb_topCustomers = pd.pivot_table(tb_movie_rating, index=['Cust_Id'], values=['Rating'], aggfunc=len).sort_values(by='Rating', ascending=False).head(5)
display(tb_topCustomers)

#'Teste'
#tt = tb_movie_rating.loc[tb_movie_rating['Cust_Id'] == 2649421] #305344
#tb_topCustomers2 = pd.pivot_table(tb_movie_rating, index=['Cust_Id'], values=['Rating'], aggfunc=len)
#tt2 = tb_topCustomers2.query('Cust_Id == [2649421]') #305344
#display(tt)
#display(tt2)

,Rating
Cust_Id,
305344,4467
387418,4422
2439493,4195
1664010,4019
2118461,3769
